# LCEL 핵심 개념

## 파이프라인(Pipeline)

**LCEL의 핵심은 파이프라인 연산자 `|`** 입니다. 이는 **Unix 파이프라인**에서 영감을 받은 개념으로, 하나의 출력이 다음 단계의 입력으로 자연스럽게 전달됩니다.

#### 파이프라인 흐름 예시

LCEL 은 동일한 방식으로 동작한다:

```python
# LCEL 체인 구성
chain = prompt_template | model | output_parser
#            ↑              ↑          ↑
# 1단계: 프롬프트 생성 → 2단계: 모델 추론 → 3단계: 결과 파싱
```

### 기본 체인의 3단계

#### PromptTemplate - 지시사항 준비
- **역할**: 사용자 입력을 AI가 이해할 수 있는 형태로 변환
- **입력**: 딕셔너리 형태의 변수들 (`{"topic": "인공지능"}`)
- **출력**: 완성된 프롬프트 문자열

#### Model - AI 처리
- **역할**: 프롬프트를 받아 AI가 답변 생성
- **입력**: 포맷팅된 프롬프트 텍스트
- **출력**: AIMessage 객체 (내용 + 메타데이터)

#### OutputParser - 결과 정리
- **역할**: AI 응답을 사용하기 쉬운 형태로 변환  
- **입력**: AIMessage 객체
- **출력**: 순수 텍스트 또는 구조화된 데이터

### 파이프라인 실행 비교

```python
# 각 단계를 개별적으로 실행하는 전통적 방식
formatted_prompt = prompt_template.format(country="대한민국")
ai_response = model.invoke(formatted_prompt)  
final_result = output_parser.parse(ai_response)

# LCEL 로 한 번에 실행하는 방식
result = chain.invoke({"country": "대한민국"})
```

LCEL 은 전체 과정을 한 번의 호출로 실행할 수 있도록 단순화한다.

---

## PromptTemplate 개요

**PromptTemplate** 은 **동적 프롬프트** 를 만들어주는 강력한 도구입니다. 마치 **편지 양식** 에 이름만 바꿔서 여러 사람에게 보내는 것과 같습니다!

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# .env 파일에 LANGCHAIN_API_KEY를 입력합니다.
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangChain-Tutorial


### PromptTemplate 의 구성 요소

**PromptTemplate** 은 **템플릿 엔진** 의 역할을 하며, 사용자 입력을 받아 **동적으로 프롬프트를 생성** 합니다.

#### 핵심 구성 요소

1. template: 실제 프롬프트 내용이 담긴 문자열 템플릿
   ```python
   template = "{country}의 수도는 어디인가요?"
   ```

2. input_variables: 템플릿에서 사용할 변수들의 이름 목록
   ```python
   # 중괄호 {} 안의 변수명들이 input_variables가 됨
   # 위 예시에서는 ["country"]가 자동으로 추출됨
   ```

#### 템플릿 변수 사용법

- **중괄호 `{}`** 안에 변수명 작성
- **여러 변수** 사용 가능: `"{name}님, {city}의 날씨는 어떤가요?"`
- **변수명은 영문자로 시작** , 숫자와 언더스코어 사용 가능

#### from_template 메서드

```python
# 간단한 방법 - from_template() 사용
prompt = PromptTemplate.from_template("{country}의 수도는 어디인가요?")

# 직접 생성
prompt = PromptTemplate(
    template="{country}의 수도는 어디인가요?",
    input_variables=["country"]
)
```

from_template 의 장점

- 자동 변수 추출: 중괄호 안의 변수를 자동 인식
- 간결한 코드: 한 줄로 템플릿 생성
- 오류 감소: 변수명 오타/누락 가능성 축소


In [3]:
# 스트리밍 출력을 위한 헬퍼 함수와 PromptTemplate 클래스 임포트
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate

### PromptTemplate 객체 생성

`from_template()` 메서드를 사용하면 간단하게 PromptTemplate 객체를 생성할 수 있다.

In [4]:
# 동적 프롬프트 템플릿 정의 - {country} 부분이 변수로 대체됨
template = "{country}의 수도는 어디인가요?"

# from_template 메서드를 이용하여 PromptTemplate 객체 생성
# 중괄호 안의 변수들이 자동으로 input_variables로 인식됨
prompt_template = PromptTemplate.from_template(template)

# 생성된 PromptTemplate 객체 확인
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
# 템플릿에 구체적인 값을 대입하여 완성된 프롬프트 생성
prompt = prompt_template.format(country="대한민국")
print(f"생성된 프롬프트: {prompt}")

생성된 프롬프트: 대한민국의 수도는 어디인가요?


In [6]:
# 다른 국가로 변경하여 프롬프트 생성 테스트
prompt = prompt_template.format(country="미국")
print(f"생성된 프롬프트: {prompt}")

생성된 프롬프트: 미국의 수도는 어디인가요?


In [7]:
# ChatOpenAI 모델 임포트 및 초기화
from langchain_openai import ChatOpenAI
import os

# OpenRouter 기반 모델 객체 생성
model = ChatOpenAI(
    temperature=0.1,
    model_name="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

In [8]:
response = model.invoke(prompt)
print(response.content)

미국의 수도는 워싱턴 D.C.(Washington, D.C.)입니다.


---

## LCEL 체인 생성

### LCEL(LangChain Expression Language) 심화

![LCEL Pipeline](./images/lcel.png)

**LCEL의 핵심은 `|` (파이프) 연산자** 입니다. 이를 통해 여러 구성 요소를 **체인처럼 연결** 하여 하나의 통합된 워크플로우를 만들 수 있습니다.

### 파이프라인 연산자의 작동 원리

```python
chain = prompt_template | model | output_parser
```

#### 데이터 흐름 과정

1. 입력: `{"topic": "인공지능"}` (딕셔너리)
2. 1단계: `prompt_template` → 완성된 프롬프트 텍스트
3. 2단계: `model` → AIMessage 객체 (AI 응답)  
4. 3단계: `output_parser` → 최종 텍스트 결과

### Unix 파이프라인과의 유사점

**Unix 명령어** 와 개념이 매우 유사합니다:

```bash
# Unix 파이프라인 예시
cat file.txt | grep "keyword" | wc -l
```

```python
# LCEL 파이프라인 예시  
chain = prompt | model | parser
```

공통점

- 왼쪽에서 오른쪽으로 데이터 흐름
- 각 단계의 출력이 다음 단계의 입력이 됨
- 모듈화: 단계별 교체/재사용 가능

### LCEL 장점

자동 최적화

- 병렬 처리: 가능한 부분은 동시에 실행
- 메모리 효율성: 중간 결과의 효율적 관리
- 스트리밍: 실시간 결과 출력 지원

개발자 친화적

- 문법이 직관적이며 데이터 흐름을 파악하기 용이
- 단계별 결과 추적이 쉬움
- 구성요소 재사용이 용이

간단한 체인을 구성한다.

In [9]:
# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template("{topic}에 대해 쉽게 설명해주세요.")

# ChatOpenAI 모델 객체 생성
import os
model = ChatOpenAI(
    temperature=0.1,
    model_name="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

# 기본 체인 구성 (출력 파서 없이) - 프롬프트와 모델만 연결
chain = prompt | model

### invoke 메서드

invoke 는 LCEL 체인을 실행하는 기본 메서드이다.

사용법
- **입력 형태**: Python 딕셔너리 `{"변수명": "값"}`
- **실행 방식**: 동기식 (결과가 나올 때까지 대기)
- **반환값**: 체인의 최종 출력 (출력 파서에 따라 달라짐)

In [10]:
# 체인 실행을 위한 입력 딕셔너리 정의
# 키는 템플릿의 변수명과 일치해야 함
input = {"topic": "인공지능 모델의 학습 원리"}

In [11]:
# LCEL 체인 실행: 프롬프트 생성 → 모델 처리 → AIMessage 반환
# invoke() 메서드로 전체 파이프라인을 한 번에 실행
result = chain.invoke(input)

# 결과 출력 (AIMessage 객체 형태로 반환됨)
result

AIMessage(content='물론입니다! 인공지능 모델, 특히 우리가 자주 듣는 ‘딥러닝’이나 ‘머신러닝’ 모델의 학습 원리를 쉽게 설명해드릴게요.\n\n---\n\n### 1. **경험을 통한 학습**\n인공지능 모델은 사람처럼 ‘경험’을 통해 학습합니다. 여기서 경험이란 ‘데이터’를 의미합니다. 예를 들어, 고양이와 개 사진을 구분하는 모델을 만들고 싶다면, 고양이 사진과 개 사진을 많이 보여줍니다.\n\n---\n\n### 2. **패턴 찾기**\n모델은 이 사진들을 보면서 고양이와 개의 특징(예: 귀 모양, 털 색깔 등)을 스스로 찾아냅니다. 이 과정을 ‘패턴 학습’이라고 합니다.\n\n---\n\n### 3. **정답과 비교하며 수정**\n모델이 사진을 보고 ‘이건 고양이야!’라고 예측하면, 실제 정답(고양이인지 개인지)과 비교합니다. 만약 틀렸다면, 모델은 ‘왜 틀렸지?’를 계산해서 내부의 규칙(수식, 가중치 등)을 조금씩 수정합니다.\n\n---\n\n### 4. **반복 학습**\n이 과정을 수천, 수만 번 반복하면서 모델은 점점 더 정확하게 고양이와 개를 구분할 수 있게 됩니다.\n\n---\n\n### 5. **새로운 데이터에 적용**\n학습이 끝난 모델은 이제 처음 보는 사진도 잘 구분할 수 있습니다. 즉, ‘경험’을 바탕으로 ‘새로운 상황’에 적용할 수 있게 된 것이죠.\n\n---\n\n#### **비유로 설명**\n마치 아이가 사과와 배를 여러 번 보고, 만지고, 먹어보면서 차이점을 배우는 것과 비슷합니다. 처음엔 헷갈릴 수 있지만, 여러 번 경험하다 보면 자연스럽게 구분할 수 있게 됩니다.\n\n---\n\n**정리하면:**  \n인공지능 모델의 학습 원리는 ‘많은 데이터를 보고, 정답과 비교하며, 틀린 부분을 고치고, 반복해서 점점 더 똑똑해지는 과정’입니다.\n\n궁금한 점이 더 있으시면 언제든 질문해주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_

### 스트리밍 출력

스트리밍 은 모델이 생성한 토큰을 순차적으로 전송하여 실시간으로 응답을 확인하는 방식이다. 긴 응답에서도 대기 시간을 단축한다.

In [12]:
# 스트리밍 방식으로 체인 실행 - 실시간으로 응답 생성 과정 확인
answer = chain.stream(input)

# langchain_teddynote의 헬퍼 함수로 스트리밍 출력을 깔끔하게 표시
stream_response(answer)

물론입니다! 인공지능 모델, 특히 딥러닝 모델의 학습 원리를 쉽게 설명해드릴게요.

---

### 1. **예시로 이해하기**
마치 어린아이가 사과와 바나나를 구분하는 법을 배우는 것과 비슷합니다.

- **데이터 제공:** 아이에게 사과와 바나나 사진을 많이 보여주면서 "이건 사과야", "이건 바나나야"라고 알려줍니다.
- **패턴 찾기:** 아이는 여러 사진을 보면서 사과와 바나나의 색, 모양 등 특징을 스스로 파악합니다.
- **예측 시도:** 이제 새로운 사진을 보여주면, 아이는 배운 특징을 바탕으로 "이건 사과야!"라고 맞추려고 합니다.
- **틀리면 수정:** 만약 틀리면, "아니야, 이건 바나나야"라고 알려주면서 다시 배우게 합니다.

---

### 2. **인공지능 모델의 학습 과정**

1. **데이터 입력:** 컴퓨터에게 많은 데이터를 줍니다. 예를 들어, 고양이와 강아지 사진과 각각의 정답(레이블)을 줍니다.
2. **예측:** 모델이 사진을 보고 "이건 고양이야" 또는 "이건 강아지야"라고 예측합니다.
3. **정답과 비교:** 모델의 예측이 실제 정답과 얼마나 다른지 확인합니다.
4. **오차 계산:** 틀린 정도(오차)를 계산합니다.
5. **모델 수정:** 오차를 줄이기 위해 모델 내부의 숫자(가중치)를 조금씩 바꿉니다.
6. **반복:** 이 과정을 수천, 수만 번 반복하면서 점점 더 정확하게 예측할 수 있게 됩니다.

---

### 3. **핵심 개념**

- **데이터와 정답(레이블)이 필요하다.**
- **예측 → 오차 계산 → 수정**의 반복이다.
- **많은 반복과 데이터가 필요하다.**

---

### 4. **비유로 정리**

인공지능 모델의 학습은,
> "실수하면서 점점 더 똑똑해지는 과정"  
이라고 생각하면 쉽습니다!

---

궁금한 점이 있으면 더 질문해 주세요!

---

## OutputParser - 출력 후처리

**OutputParser** 는 AI 의 복잡한 응답을 **사용하기 쉬운 형태로 변환** 해주는 마지막 단계입니다.

In [13]:
# 문자열 출력 파서 임포트
from langchain_core.output_parsers import StrOutputParser

# StrOutputParser 객체 생성 - AIMessage에서 순수 텍스트만 추출
output_parser = StrOutputParser()

### 완전한 체인 구성

이제 **3단계 파이프라인** 을 완성해봅시다: **PromptTemplate → Model → OutputParser**

In [14]:
# 완전한 LCEL 체인 구성: 프롬프트 → 모델 → 출력 파서
# 이제 결과가 AIMessage가 아닌 순수 문자열로 반환됨
chain = prompt | model | output_parser

In [15]:
# 완성된 체인으로 invoke 실행 - 이제 순수 문자열이 반환됨
input = {"topic": "인공지능 모델의 학습 원리"}
result = chain.invoke(input)

# 결과 출력 (이제 문자열 형태로 깔끔하게 출력됨)
print("=== 완성된 체인 결과 ===")
print(result)

=== 완성된 체인 결과 ===
물론입니다! 인공지능 모델, 특히 우리가 자주 듣는 ‘딥러닝’이나 ‘머신러닝’ 모델의 학습 원리를 쉽게 설명해드릴게요.

---

### 1. **경험을 통한 학습**
인공지능 모델은 사람처럼 ‘경험’을 통해 학습합니다. 여기서 경험이란 ‘데이터’를 의미합니다. 예를 들어, 고양이와 개 사진을 구분하는 모델을 만든다고 할 때, 수많은 고양이와 개 사진(데이터)을 보여주며 학습시킵니다.

---

### 2. **예측과 정답 비교**
모델은 처음에는 무작위로 예측을 합니다. 예를 들어, 어떤 사진을 보고 ‘이건 고양이야!’라고 예측합니다. 그 다음, 실제 정답(예: ‘이건 개야!’)과 비교해서 얼마나 틀렸는지 확인합니다.

---

### 3. **오차(실수) 계산**
예측과 정답이 다르면, 그 차이(오차)를 계산합니다. 이 오차가 작을수록 모델이 잘 맞춘 것이고, 오차가 크면 많이 틀린 것입니다.

---

### 4. **오차를 줄이기 위한 조정**
모델은 오차를 줄이기 위해 내부의 ‘가중치’라는 값을 조금씩 바꿉니다. 이 과정을 수천, 수만 번 반복하면서 점점 더 정확하게 예측할 수 있게 됩니다. 마치 사람이 틀린 문제를 반복해서 풀면서 점점 더 잘 맞추게 되는 것과 비슷합니다.

---

### 5. **반복 학습**
이 과정을 데이터 전체에 대해 여러 번 반복합니다. 반복할수록 모델은 점점 더 똑똑해집니다.

---

#### **정리**
- 데이터를 많이 보여주고,
- 예측을 해보고,
- 정답과 비교해서,
- 틀린 만큼 내부를 조정하고,
- 이 과정을 반복해서,
- 점점 더 정확하게 예측하게 되는 것!

이것이 인공지능 모델의 기본적인 학습 원리입니다.

궁금한 점이 더 있으시면 언제든 질문해주세요!


In [16]:
# 완성된 체인으로 스트리밍 실행
answer = chain.stream(input)

print("=== 스트리밍 출력 ===")
# 실시간으로 문자열이 생성되는 과정을 관찰
stream_response(answer)

=== 스트리밍 출력 ===
물론입니다! 인공지능 모델, 특히 많이 쓰이는 **딥러닝** 모델의 학습 원리를 쉽게 설명해드릴게요.

---

### 1. **모델은 뇌의 신경망을 흉내냅니다**
- 인공지능 모델은 사람 뇌의 신경망(뉴런)을 본떠 만든 구조입니다.
- 입력(예: 사진, 글 등)을 받아서 여러 층을 거치며 결과(예: 고양이/강아지, 긍정/부정 등)를 내놓습니다.

---

### 2. **예시로 설명해볼게요**
- 예를 들어, 고양이와 강아지 사진을 구분하는 모델을 만든다고 해봅시다.
- 고양이 사진에는 '고양이', 강아지 사진에는 '강아지'라는 정답(라벨)을 붙여서 모델에 보여줍니다.

---

### 3. **모델이 예측을 합니다**
- 모델은 처음에는 아무것도 모릅니다. 그래서 엉뚱하게 예측할 수 있습니다.
- 예를 들어, 고양이 사진을 보고 '강아지'라고 답할 수도 있습니다.

---

### 4. **정답과 비교해서 틀린 정도(오차)를 계산합니다**
- 모델의 답과 실제 정답을 비교해서 얼마나 틀렸는지(오차)를 계산합니다.

---

### 5. **오차를 줄이도록 모델을 조금씩 수정합니다**
- 오차가 작아지도록 모델 내부의 숫자(가중치)를 조금씩 바꿉니다.
- 이 과정을 **역전파(Backpropagation)**와 **경사하강법(Gradient Descent)**이라는 방법으로 반복합니다.

---

### 6. **이 과정을 수천~수백만 번 반복합니다**
- 사진을 계속 보여주고, 예측하고, 오차를 계산하고, 모델을 수정하는 과정을 반복합니다.
- 반복할수록 모델은 점점 더 정답에 가까운 예측을 하게 됩니다.

---

### 7. **학습이 끝나면, 새로운 사진도 잘 맞춥니다**
- 이제 모델은 처음 보는 사진도 고양이인지 강아지인지 잘 맞출 수 있게 됩니다.

---

#### **정리**
- 인공지능 모델은 **입력과 정답을 많이 보여주면서**, **틀린 만큼 내부를 조금씩 수정**해서, **정답을 잘 맞추도록 배우는 것**입

---

## 실습: 영어 회화 튜터 예제

이제 배운 내용을 활용해서 **실용적인 영어 학습 도우미** 를 만들어봅니다. 

### 프로젝트 개요

- **목표**: 상황별 영어 회화 생성 + 한글 번역 제공
- **특징**: 체계적인 포맷으로 학습 효과 극대화
- **활용**: 다양한 상황에 맞는 영어 표현 학습

In [17]:
# 전문적인 영어 회화 튜터 프롬프트 템플릿 설계
template = """You are an experienced English conversation teacher with 10 years of expertise.
Create practical English conversations for the given situation with Korean translations.
Please follow the FORMAT exactly as shown below.

#SITUATION:
{question}

#FORMAT:
- English Conversation:
- Korean Translation:
- Useful Expressions:
- Cultural Notes (if applicable):
"""

# 개선된 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 모델 객체 생성 (OpenRouter 사용)
import os
model = ChatOpenAI(
    temperature=0.1,
    model_name="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

# 문자열 출력 파서 생성
output_parser = StrOutputParser()

In [18]:
# 영어 회화 튜터 체인 구성
# 프롬프트 → 모델 → 출력 파서의 완전한 파이프라인
chain = prompt | model | output_parser

In [19]:
# 첫 번째 상황: 식당에서 음식 주문하기
situation_1 = "저는 식당에 가서 음식을 주문하고 싶어요"

print("🍽️ === 식당 주문 상황 ===")
print(chain.invoke({"question": situation_1}))

🍽️ === 식당 주문 상황 ===
#SITUATION:  
저는 식당에 가서 음식을 주문하고 싶어요

- English Conversation:  
Waiter: Hello! Welcome to our restaurant. How many people are in your party?  
Customer: Hi! Just one, please.  
Waiter: Right this way. Here is the menu.  
Customer: Thank you.  
Waiter: Are you ready to order?  
Customer: Yes, I’d like the chicken pasta, please.  
Waiter: Would you like anything to drink?  
Customer: Just water, please.  
Waiter: Sure. Your order will be ready soon.

- Korean Translation:  
웨이터: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 몇 분이세요?  
손님: 안녕하세요! 저 혼자예요.  
웨이터: 이쪽으로 오세요. 메뉴판입니다.  
손님: 감사합니다.  
웨이터: 주문하시겠어요?  
손님: 네, 치킨 파스타로 주세요.  
웨이터: 음료는 무엇으로 드릴까요?  
손님: 물로 주세요.  
웨이터: 알겠습니다. 곧 준비해드릴게요.

- Useful Expressions:  
- I’d like to order, please. (주문하고 싶어요.)  
- Can I see the menu? (메뉴판을 볼 수 있을까요?)  
- What do you recommend? (추천 메뉴가 있나요?)  
- I’ll have the ____, please. (____로 할게요.)  
- Can I get this to go? (이거 포장해 주실 수 있나요?)  
- Could I have the bill, please? (계산서 주세요.)

- Cultural Notes (if

In [20]:
# 두 번째 상황: 스트리밍으로 실시간 학습 경험
situation_2 = "미국에서 피자 주문"

print("🍕 === 미국 피자 주문 상황 (스트리밍) ===")
# 스트리밍으로 영어 회화가 실시간으로 생성되는 과정 관찰
answer = chain.stream({"question": situation_2})
stream_response(answer)

🍕 === 미국 피자 주문 상황 (스트리밍) ===
#SITUATION:  
미국에서 피자 주문

- English Conversation:  
Clerk: Hi, welcome to Pizza Palace! How can I help you today?  
Customer: Hi, I’d like to order a large pepperoni pizza for takeout, please.  
Clerk: Sure! Would you like any extra toppings or sides with that?  
Customer: Yes, can I add mushrooms and get a side of garlic bread?  
Clerk: Absolutely. Anything to drink?  
Customer: I’ll have a bottle of Coke, please.  
Clerk: Great! Your total comes to $18.50. It’ll be ready in about 20 minutes.  
Customer: Thank you! I’ll be back soon to pick it up.  
Clerk: You’re welcome! See you soon.

- Korean Translation:  
점원: 안녕하세요, 피자 팰리스에 오신 걸 환영합니다! 무엇을 도와드릴까요?  
손님: 안녕하세요, 포장용으로 라지 페퍼로니 피자 하나 주문하고 싶어요.  
점원: 네! 추가 토핑이나 사이드 메뉴도 필요하신가요?  
손님: 네, 버섯 추가하고 마늘빵도 하나 주세요.  
점원: 물론이죠. 음료도 필요하신가요?  
손님: 콜라 한 병 주세요.  
점원: 알겠습니다! 총 금액은 18달러 50센트입니다. 약 20분 후에 준비됩니다.  
손님: 감사합니다! 곧 가지러 올게요.  
점원: 네, 감사합니다! 곧 뵙겠습니다.

- Useful Expressions:  
- I’d like to order a (large/small/med

In [21]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

#SITUATION:  
저는 식당에 가서 음식을 주문하고 싶어요

- English Conversation:  
Waiter: Hello! Welcome to our restaurant. How many people are in your party?  
Customer: Hi! Just one, please.  
Waiter: Right this way. Here is the menu.  
Customer: Thank you.  
Waiter: Are you ready to order?  
Customer: Yes, I’d like the grilled chicken, please.  
Waiter: Would you like anything to drink?  
Customer: Yes, I’ll have an iced tea.  
Waiter: Great. Your order will be ready soon.  
Customer: Thank you!

- Korean Translation:  
웨이터: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 몇 분이세요?  
손님: 안녕하세요! 저 혼자예요.  
웨이터: 이쪽으로 오세요. 여기 메뉴판입니다.  
손님: 감사합니다.  
웨이터: 주문하시겠어요?  
손님: 네, 그릴 치킨 주세요.  
웨이터: 음료는 무엇으로 드릴까요?  
손님: 아이스티 주세요.  
웨이터: 알겠습니다. 곧 준비해드릴게요.  
손님: 감사합니다!

- Useful Expressions:  
- How many people are in your party? (몇 분이세요?)  
- Are you ready to order? (주문하시겠어요?)  
- I’d like [menu item], please. ([메뉴] 주세요.)  
- Would you like anything to drink? (음료는 무엇으로 드릴까요?)  
- Can I have the bill, please? (계산서 주세요.)

- Cultural Notes (

In [22]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

#SITUATION:  
미국에서 피자 주문

- English Conversation:  
Clerk: Hi, welcome to Joe’s Pizza! How can I help you today?  
Customer: Hi, I’d like to order a large pepperoni pizza, please.  
Clerk: Sure! Would you like any extra toppings or sides with that?  
Customer: Yes, can I add mushrooms and get a side of garlic bread?  
Clerk: Absolutely. Would you like your pizza for pickup or delivery?  
Customer: Delivery, please.  
Clerk: Great! Can I have your address and phone number?  
Customer: Sure, it’s 123 Maple Street, and my number is 555-1234.  
Clerk: Thank you. Your total is $18.50. It should arrive in about 30 minutes.  
Customer: Perfect, thank you!  
Clerk: You’re welcome. Enjoy your meal!

- Korean Translation:  
점원: 안녕하세요, Joe’s Pizza에 오신 것을 환영합니다! 무엇을 도와드릴까요?  
손님: 안녕하세요, 라지 페퍼로니 피자 하나 주문하고 싶어요.  
점원: 네! 추가 토핑이나 사이드 메뉴도 필요하신가요?  
손님: 네, 버섯 추가하고 마늘빵도 하나 주세요.  
점원: 알겠습니다. 피자는 포장해 가실 건가요, 배달해 드릴까요?  
손님: 배달로 해주세요.  
점원: 네! 주소와 전화번호 알려주시겠어요?  
손님: 네, 주소는 메이플 스트리트 123번지이고, 전화번호는 555-1234